In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools
from datetime import datetime

In [2]:
dataset=pd.read_csv("breast-std.csv")
dataset=dataset.sort_values(by=['class'], ascending=False)


In [3]:
X = dataset.columns[1:]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [4]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [5]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        mse=mean_squared_error(y, y_predict, squared=False)
        acc=correct/len(y_predict)
        print("Accuracy",correct/len(y_predict))
        print("Errore quadratico medio: ", mse)
        return mse
    
    def accuracy(self, X, y):
        y_predict=self.predict(X)
        correct = np.sum(y_predict == y)
        acc=correct/len(y_predict)
        return acc
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
                #Valori di membership?
                #print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))
    

In [6]:
C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
err = []
acc = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
print(datetime.now())
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    print("\nBest model", best_C, best_sigma)
    best_model.fit(X_train, y_train)
    err.append(best_model.score(X_validation, y_validation))
    acc.append(best_model.accuracy(X_validation, y_validation))
print(datetime.now())    

2020-10-04 10:45:55.479543
     pcost       dcost       gap    pres   dres
 0: -3.8267e+01 -1.4011e+01  2e+03  5e+01  1e-16
 1: -7.0334e+00 -6.8577e+00  2e+02  3e+00  2e-16
 2: -1.0046e+00 -6.0197e+00  9e+00  9e-02  9e-16
 3: -7.6938e-01 -1.5645e+00  8e-01  2e-16  6e-16
 4: -7.7646e-01 -7.9450e-01  2e-02  2e-15  2e-16
 5: -7.7714e-01 -7.8509e-01  8e-03  3e-16  9e-17
 6: -7.7753e-01 -7.8128e-01  4e-03  6e-16  2e-16
 7: -7.7774e-01 -7.7801e-01  3e-04  2e-15  1e-16
 8: -7.7776e-01 -7.7776e-01  5e-06  4e-15  2e-16
 9: -7.7776e-01 -7.7776e-01  5e-08  4e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.8267e+01 -1.4011e+01  2e+03  5e+01  1e-16
 1: -7.0334e+00 -6.8577e+00  2e+02  3e+00  2e-16
 2: -1.0046e+00 -6.0197e+00  9e+00  9e-02  9e-16
 3: -7.6938e-01 -1.5645e+00  8e-01  2e-16  6e-16
 4: -7.7646e-01 -7.9450e-01  2e-02  2e-15  2e-16
 5: -7.7714e-01 -7.8509e-01  8e-03  3e-16  9e-17
 6: -7.7753e-01 -7.8128e-01  4e-03  6e-16  2e-16
 7: -7.7774e-01 -7.7

47 out of 86 predictions correct
Accuracy 0.5465116279069767
Errore quadratico medio:  1.3468309056344427
     pcost       dcost       gap    pres   dres
 0: -7.5821e+01 -2.4565e+01  2e+03  4e+01  4e-16
 1: -1.9302e+01 -8.5253e+00  1e+02  3e+00  5e-16
 2: -2.7293e+00 -7.0095e+00  6e+00  4e-02  1e-15
 3: -2.4704e+00 -2.7132e+00  2e-01  2e-16  9e-16
 4: -2.4810e+00 -2.5285e+00  5e-02  3e-17  4e-16
 5: -2.4835e+00 -2.4977e+00  1e-02  3e-17  3e-16
 6: -2.4857e+00 -2.4873e+00  2e-03  3e-17  4e-16
 7: -2.4860e+00 -2.4862e+00  1e-04  4e-16  4e-16
 8: -2.4860e+00 -2.4861e+00  5e-06  2e-16  4e-16
 9: -2.4860e+00 -2.4860e+00  2e-07  1e-15  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.5821e+01 -2.4565e+01  2e+03  4e+01  4e-16
 1: -1.9302e+01 -8.5253e+00  1e+02  3e+00  5e-16
 2: -2.7293e+00 -7.0095e+00  6e+00  4e-02  1e-15
 3: -2.4704e+00 -2.7132e+00  2e-01  2e-16  9e-16
 4: -2.4810e+00 -2.5285e+00  5e-02  3e-17  4e-16
 5: -2.4835e+00 -2.4977e+00  1e-02  3e-

85 out of 85 predictions correct
Accuracy 1.0
Errore quadratico medio:  0.0
     pcost       dcost       gap    pres   dres
 0: -1.2536e+02 -1.2448e+01  2e+03  5e+01  3e-15
 1: -1.1165e+01 -8.4749e+00  1e+02  2e+00  3e-15
 2: -2.4742e+00 -7.3164e+00  6e+00  3e-02  2e-15
 3: -2.4460e+00 -2.9182e+00  5e-01  2e-03  7e-16
 4: -2.4683e+00 -2.5701e+00  1e-01  1e-04  5e-16
 5: -2.4872e+00 -2.5085e+00  2e-02  2e-05  5e-16
 6: -2.4921e+00 -2.4972e+00  5e-03  3e-06  6e-16
 7: -2.4933e+00 -2.4948e+00  1e-03  6e-07  6e-16
 8: -2.4938e+00 -2.4941e+00  4e-04  8e-08  6e-16
 9: -2.4939e+00 -2.4940e+00  1e-04  1e-08  6e-16
10: -2.4939e+00 -2.4939e+00  1e-05  1e-15  7e-16
11: -2.4939e+00 -2.4939e+00  9e-07  3e-17  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.2536e+02 -1.2448e+01  2e+03  5e+01  3e-15
 1: -1.1165e+01 -8.4749e+00  1e+02  2e+00  3e-15
 2: -2.4742e+00 -7.3164e+00  6e+00  3e-02  2e-15
 3: -2.4460e+00 -2.9182e+00  5e-01  2e-03  7e-16
 4: -2.4683e+00 -2.5

85 out of 85 predictions correct
Accuracy 1.0
Errore quadratico medio:  0.0
     pcost       dcost       gap    pres   dres
 0: -1.4992e+02 -9.6686e+00  2e+03  5e+01  1e-14
 1: -8.6878e+00 -7.8543e+00  8e+01  2e+00  1e-14
 2: -2.4099e+00 -6.8961e+00  5e+00  4e-03  2e-15
 3: -2.4902e+00 -2.6662e+00  2e-01  2e-04  7e-16
 4: -2.4953e+00 -2.5302e+00  4e-02  3e-05  6e-16
 5: -2.4967e+00 -2.5153e+00  2e-02  6e-06  6e-16
 6: -2.4992e+00 -2.5015e+00  2e-03  7e-07  7e-16
 7: -2.4996e+00 -2.5001e+00  6e-04  1e-07  6e-16
 8: -2.4996e+00 -2.5000e+00  4e-04  5e-08  4e-16
 9: -2.4996e+00 -2.4999e+00  3e-04  4e-08  4e-16
10: -2.4996e+00 -2.4999e+00  3e-04  3e-08  4e-16
11: -2.4997e+00 -2.4998e+00  7e-05  6e-09  7e-16
12: -2.4997e+00 -2.4997e+00  2e-05  9e-10  9e-16
13: -2.4997e+00 -2.4997e+00  9e-06  2e-10  1e-15
14: -2.4997e+00 -2.4997e+00  2e-06  3e-11  7e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4992e+02 -9.6686e+00  2e+03  5e+01  1e-14
 1: -8.6878e+00 -7.8

0 out of 86 predictions correct
Accuracy 0.0
Errore quadratico medio:  2.0
     pcost       dcost       gap    pres   dres
 0: -1.2991e+02 -7.2513e+00  2e+03  5e+01  1e-14
 1: -3.1906e+00 -7.2102e+00  3e+01  5e-01  1e-14
 2: -1.7699e+00 -4.8701e+00  3e+00  1e-02  1e-15
 3: -1.7189e+00 -1.7649e+00  5e-02  8e-05  1e-15
 4: -1.7200e+00 -1.7205e+00  5e-04  8e-07  6e-16
 5: -1.7200e+00 -1.7200e+00  7e-06  1e-08  6e-16
 6: -1.7200e+00 -1.7200e+00  1e-06  2e-09  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.2991e+02 -7.2513e+00  2e+03  5e+01  1e-14
 1: -3.1906e+00 -7.2102e+00  3e+01  5e-01  1e-14
 2: -1.7699e+00 -4.8701e+00  3e+00  1e-02  1e-15
 3: -1.7189e+00 -1.7649e+00  5e-02  8e-05  1e-15
 4: -1.7200e+00 -1.7205e+00  5e-04  8e-07  6e-16
 5: -1.7200e+00 -1.7200e+00  7e-06  1e-08  6e-16
 6: -1.7200e+00 -1.7200e+00  1e-06  2e-09  5e-16
Optimal solution found.
47 out of 86 predictions correct
Accuracy 0.5465116279069767
Errore quadratico medio:  1.346830

...

63 out of 86 predictions correct
Accuracy 0.7325581395348837
Errore quadratico medio:  1.034295625950562
     pcost       dcost       gap    pres   dres
 0:  1.7794e+08 -9.4245e+09  1e+10  2e-17  1e-10
 1:  4.5205e+07 -1.2603e+09  1e+09  1e-10  1e-10
 2:  6.7748e+06 -1.5131e+08  2e+08  5e-10  1e-10
 3:  7.0531e+05 -3.8883e+07  4e+07  3e-10  1e-10
 4: -8.7099e+05 -1.1673e+07  1e+07  5e-10  1e-10
 5: -1.3020e+06 -3.5589e+06  2e+06  7e-10  1e-10
 6: -1.6534e+06 -2.5362e+06  9e+05  6e-10  2e-10
 7: -1.7594e+06 -2.3358e+06  6e+05  2e-10  2e-10
 8: -1.8493e+06 -2.1247e+06  3e+05  2e-10  2e-10
 9: -1.9103e+06 -2.0078e+06  1e+05  2e-10  2e-10
10: -1.9323e+06 -1.9654e+06  3e+04  7e-10  2e-10
11: -1.9421e+06 -1.9478e+06  6e+03  7e-10  2e-10
12: -1.9441e+06 -1.9447e+06  5e+02  5e-10  2e-10
13: -1.9443e+06 -1.9444e+06  7e+01  1e-10  2e-10
14: -1.9443e+06 -1.9443e+06  4e+00  3e-10  2e-10
15: -1.9443e+06 -1.9443e+06  1e-01  7e-10  2e-10
Optimal solution found.
     pcost       dcost       gap    pre

73 out of 86 predictions correct
Accuracy 0.8488372093023255
Errore quadratico medio:  0.777593185920953
     pcost       dcost       gap    pres   dres
 0:  7.3677e+06 -3.5550e+09  4e+09  5e-10  1e-10
 1:  5.8240e+06 -1.5183e+08  2e+08  6e-10  2e-10
 2:  5.3134e+05 -7.0844e+07  7e+07  5e-10  1e-10
 3: -1.2076e+06 -1.5614e+07  1e+07  6e-10  1e-10
 4: -1.5748e+06 -4.0183e+06  2e+06  2e-10  1e-10
 5: -1.7979e+06 -3.4119e+06  2e+06  7e-10  2e-10
 6: -2.0271e+06 -2.8003e+06  8e+05  2e-16  2e-10
 7: -2.1423e+06 -2.4591e+06  3e+05  2e-09  2e-10
 8: -2.1646e+06 -2.3952e+06  2e+05  1e-09  2e-10
 9: -2.1776e+06 -2.3761e+06  2e+05  1e-09  2e-10
10: -2.2208e+06 -2.2648e+06  4e+04  1e-09  2e-10
11: -2.2282e+06 -2.2492e+06  2e+04  5e-10  2e-10
12: -2.2305e+06 -2.2454e+06  1e+04  3e-10  2e-10
13: -2.2337e+06 -2.2389e+06  5e+03  9e-10  2e-10
14: -2.2348e+06 -2.2373e+06  2e+03  3e-10  2e-10
15: -2.2354e+06 -2.2364e+06  9e+02  7e-10  2e-10
16: -2.2356e+06 -2.2362e+06  6e+02  6e-10  2e-10
17: -2.2358e+0

     pcost       dcost       gap    pres   dres
 0: -1.4972e+06 -3.3243e+09  3e+09  8e-10  1e-10
 1: -1.4981e+06 -3.6934e+07  4e+07  6e-10  2e-10
 2: -1.5724e+06 -3.9763e+06  2e+06  3e-10  2e-10
 3: -1.7539e+06 -3.8224e+06  2e+06  8e-16  2e-10
 4: -1.8930e+06 -3.7912e+06  2e+06  2e-10  2e-10
 5: -2.1897e+06 -3.1234e+06  9e+05  2e-10  2e-10
 6: -2.3773e+06 -2.7806e+06  4e+05  6e-10  2e-10
 7: -2.4676e+06 -2.5242e+06  6e+04  1e-09  2e-10
 8: -2.4789e+06 -2.4885e+06  1e+04  8e-10  2e-10
 9: -2.4803e+06 -2.4860e+06  6e+03  1e-10  2e-10
10: -2.4815e+06 -2.4833e+06  2e+03  2e-16  2e-10
11: -2.4818e+06 -2.4828e+06  9e+02  2e-10  2e-10
12: -2.4820e+06 -2.4825e+06  4e+02  3e-10  2e-10
13: -2.4821e+06 -2.4823e+06  3e+02  1e-09  2e-10
14: -2.4822e+06 -2.4823e+06  1e+02  3e-10  2e-10
15: -2.4822e+06 -2.4822e+06  1e+01  2e-09  2e-10
16: -2.4822e+06 -2.4822e+06  5e-01  1e-09  2e-10
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4972e+06 -3.3243e+09  3e+09  8e-10  1e-1

46 out of 86 predictions correct
Accuracy 0.5348837209302325
Errore quadratico medio:  1.363988678940947
     pcost       dcost       gap    pres   dres
 0: -1.5908e+06 -3.3230e+09  3e+09  3e-10  1e-10
 1: -1.5917e+06 -3.5737e+07  3e+07  3e-10  2e-10
 2: -1.6726e+06 -2.8658e+06  1e+06  3e-10  2e-10
 3: -2.5093e+06 -2.5968e+06  9e+04  2e-10  2e-10
 4: -2.5180e+06 -2.5275e+06  1e+04  1e-10  2e-10
 5: -2.5190e+06 -2.5228e+06  4e+03  5e-10  2e-10
 6: -2.5195e+06 -2.5204e+06  9e+02  1e-10  2e-10
 7: -2.5196e+06 -2.5198e+06  2e+02  6e-10  2e-10
 8: -2.5196e+06 -2.5197e+06  6e+01  1e-09  2e-10
 9: -2.5196e+06 -2.5196e+06  2e+01  6e-10  2e-10
10: -2.5196e+06 -2.5196e+06  9e+00  3e-10  2e-10
11: -2.5196e+06 -2.5196e+06  4e+00  9e-10  2e-10
12: -2.5196e+06 -2.5196e+06  2e+00  1e-09  2e-10
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5908e+06 -3.3230e+09  3e+09  3e-10  1e-10
 1: -1.5917e+06 -3.5737e+07  3e+07  3e-10  2e-10
 2: -1.6726e+06 -2.8658e+06  1e+06  3e-1

2020-10-04 11:34:38.062266


In [7]:
print(np.mean(err), np.mean(acc))

1.0423205774579156 0.7265032622112502


In [8]:
print(err, acc)

[1.093737483239461, 0.927261065780371, 0.9472044455566301, 1.1106956648034079, 1.1327042279097082] [0.7009345794392523, 0.7850467289719626, 0.7757009345794392, 0.6915887850467289, 0.6792452830188679]
